This notebook is for performing AgreementPred prediction on query compounds

See caption in the codes for more details.

In [1]:
import os
import sys
import glob

import pandas as pd
import numpy as np
import json

from sklearn.metrics.pairwise import cosine_similarity
from scipy import stats
from scipy.spatial.distance import jaccard
import math

from tqdm import tqdm

import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
main_folder = "/home/data/grd22/AgreementPred_Codes"

In [19]:
#AgreementPred prediction (22 representations)

cmpd_annot = pd.read_csv(f"{main_folder}/Data/Annotations/Annotated-Compound_Annotations.csv")

fps = pd.read_csv(f"{main_folder}/Data/29FP_lst.csv")
fp_dict = {f:n for f,n in zip(fps['fp'], fps['name'])}
fp_lst = list(fp_dict.keys())

#Read 29 representations of query compounds
folder = f"{main_folder}/MSC_Query_Df"
Sim_dfs = glob.glob(f"{folder}/*")
df_dict = {}
df_lst = []
for f in Sim_dfs:
    f2 = f.replace(f'{folder}/','')
    name = f2.replace('.csv', '')

    temp = pd.read_csv(f)
    temp = temp.drop_duplicates('cid').reset_index(drop=True)

    temp = temp.reset_index(drop=True)
    temp['MSCs'] = temp['MSCs'].apply(json.loads)
    temp['similarities'] = temp['similarities'].apply(json.loads)

    df_dict[name] = temp
    df_lst.append(name)


#Prediction

#N of MSCs
length = 1

#Treshold
threshold = 0.1


pred_res = []
all_preds = []

#Selected representations (22)

fp_lst = list(set(fp_lst)-set(['rdkitTT','jmapASP', 'cdkDaylight', 'cdkFP', 'MorganFP', 'minhashMHFP', 'jmapPH2']))
fp_num = len(fp_lst)
print(f"Using {fp_num} representations: {fp_lst}")

# Iterate over unique tcid values

folder_path = main_folder

testset = df_dict[df_lst[0]]['cid'].tolist()
cmpdnames = df_dict[df_lst[0]]['cmpdname'].tolist()

for tcid, cname in zip(testset, cmpdnames):

    row = {'test_cid': tcid,
        'cmpdname': cname}


    sel = []
    sim = []
    for name in fp_lst:

        sim_coms = df_dict[name]
        sim_coms = sim_coms[['cid', 'cmpdname', 'MSCs', 'similarities']]
        
        com_lst = sim_coms.loc[sim_coms['cid'] == tcid, 'MSCs'].values[0]
        sim_lst = sim_coms.loc[sim_coms['cid'] == tcid, 'similarities'].values[0]

        sel.extend(com_lst[:length])
        sim.extend(sim_lst[:length])

    row['fp_list'] = str(fp_lst)
    row['similar_compounds'] = str(list(sel))
    row['compound_num'] = len(set(sel))

    pred_res.append(row)


    pred_df = pd.DataFrame()
    for com in sel:
        t1 = cmpd_annot[cmpd_annot['cid']==com]
        pred_df = pd.concat([pred_df, t1])

    pred = pred_df['annot'].value_counts().index.tolist()

    for pre in pred:
        cnt = pred_df['annot'].value_counts()[pre]
        agree_score = cnt / (fp_num*length)
        if agree_score > 1:
            agree_score = 1
            
        r = {
            'test_cid': tcid,
            'cmpdname': cname,
            'prediction': pre,
            'agreement score': round(agree_score, 2),
        }
        all_preds.append(r)

final = pd.DataFrame(all_preds)
final = final[final['agreement score'] > threshold]


pd.DataFrame(pred_res).to_csv(f"{main_folder}/Results/Query_similar_compounds.csv", index=False)
final.to_csv(f"{main_folder}/Results/Query_Threshold_{threshold}_Predictions.csv", index=False)


Using 22 representations: ['jmapLSTAR', 'rdkitECFP', 'ExtFP', 'KRFP', 'rdkitAvalon', 'maccsFP', 'rdkitRDKIT', 'LingoFP', 'jmapPH3', 'PubchemFP', 'CircFP', 'HybridFP', 'InfoGraphFeature', 'jmapRAD2D', 'GraphFP', 'EStateFP', 'SPFP', 'rdkitAP', 'minhashMAP4', 'AP2DFP', 'rdkitFCFP', 'jmapDFS']
